# Playing Card Generator

## Imports

In [ ]:
import os
import logging

import prettymaps

import pandas as pd
from PIL import Image
from svglib.svglib import svg2rlg

from reportlab.pdfgen import canvas
from reportlab.graphics import renderPDF
from reportlab.lib.colors import HexColor

## Configuration

In [ ]:
# Dimension of playing card
CARD_WIDTH, CARD_HEIGHT = 822, 1122

# Radius parameter for map generation
MAP_RADIUS = 400

# Set amount of cropping
BORDER_CROP = 210

# Output directory
OUTPUT_DIR = "cards_berlin"

# Temporary directory and filenames
TMP_DIR = "tmp"
MAP_FILE = f"{TMP_DIR}/map.png"
CROP_FILE = f"{TMP_DIR}/crop.png"

# Assets
ASSETS_DIR = "assets"
LOCATIONS_FILE = f"{ASSETS_DIR}/locations_berlin.csv"
MASK_FILE = f"{ASSETS_DIR}/mask.svg"

# Text color & size
PRIMARY_TEXT_COLOR = 0x2F3737
PRIMARY_FONT_TYPE = "Times-Bold"
SECONDARY_TEXT_COLOR = 0x646464
SECONDARY_FONT_TYPE = "Times-Italic"

In [ ]:
# Suppress warnings from matplotlib
logging.getLogger('matplotlib').setLevel(logging.ERROR)

## Helper functions

In [ ]:
def get_primary_font_size(text):
    font_sizes = {
        (0, 15): 80,
        (15, 19): 70,
        (19, 25): 60,
        (25, 30): 50,
        (30, 35): 45
    }
    for (start, end), size in font_sizes.items():
        if start <= len(text) < end:
            return size
    return 40

def get_secondary_font_size(text):
    font_sizes = {
        (0, 30): 50,
        (30, 40): 42,
        (40, 45): 35,
    }
    for (start, end), size in font_sizes.items():
        if start <= len(text) < end:
            return size
    return 32

In [ ]:
def generate_playing_card(location, title, subtitle=None):
    """
    Generate a playing card with a map, title, and subtitle.

    Args:
        location (str): Location for the map.
        title (str): Title of the playing card.
        subtitle (str): Subtitle of the playing card.

    Returns:
        None
    """
    
    # Generate map plot
    plot = prettymaps.plot(location, radius=MAP_RADIUS, show=False)

    # Create tmp directory, if it doesn't exist
    if not os.path.exists(TMP_DIR):
        os.makedirs(TMP_DIR)

    # Create output directory, if it doesn't exist
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)

    # Save map plot
    plot.fig.savefig(MAP_FILE, transparent=False, dpi=200)

    # Crop image border
    im = Image.open(MAP_FILE)
    im = im.crop((BORDER_CROP, BORDER_CROP, im.width - BORDER_CROP, im.height - BORDER_CROP))

    # Crop to fit card size
    map_scale = CARD_HEIGHT / im.height
    margin_x = (im.width * map_scale - CARD_WIDTH) / (2 * map_scale)

    im = im.crop((margin_x, 0, im.width - margin_x, im.height))
    im.save(CROP_FILE)

    # Set filename and primary text offset
    if subtitle:
        filename = f"{title}-{subtitle}"
        offset = 200
    else:
        filename = title
        offset = 230

    filename = "".join([c for c in filename if c.isalpha() or c.isdigit() or c == '-']).rstrip()

    # Open canvas image
    c = canvas.Canvas(f"{OUTPUT_DIR}/{filename}.pdf", pagesize=(CARD_WIDTH, CARD_HEIGHT))
    
    # Add map background
    c.drawImage(CROP_FILE, 0, 0, CARD_WIDTH, CARD_HEIGHT)

    # Add overlay
    renderPDF.draw(svg2rlg(MASK_FILE), c, 0, 0)

    # Add primary text
    c.setFont(PRIMARY_FONT_TYPE, get_primary_font_size(title))
    c.setFillColor(HexColor(PRIMARY_TEXT_COLOR))
    c.drawCentredString(CARD_WIDTH / 2, CARD_HEIGHT - offset, title)

    # Add secondary text
    if subtitle:
        c.setFont(SECONDARY_FONT_TYPE, get_secondary_font_size(subtitle))
        c.setFillColor(HexColor(SECONDARY_TEXT_COLOR))
        c.drawCentredString(CARD_WIDTH / 2, CARD_HEIGHT - 270, subtitle)
    
    # Close canvas image
    c.save()


## Generate all playingcards

In [ ]:
def generate_all_playing_cards():
    """
    Generates playing cards for each location in the CSV file.
    The CSV file should contain columns for 'Generated', 'OSM', 'Name', and 'Beschreibung'.
    For each location, if 'Generated' is not True and 'OSM' is not null, a card is generated.
    After generating the card, the 'Generated' column is updated to True in the CSV file.
    """
    # Read csv file in form of list of lists
    locations = pd.read_csv(LOCATIONS_FILE)

    for index, row in locations.iterrows():
        generated = row['Generated']
        location = row['OSM']
        name = row['Name']
        description = row['Beschreibung']

        if pd.notnull(generated) and not bool(generated) and pd.notnull(location):
            print("Generating card for", name, "...")

            if pd.notnull(description):
                generate_playing_card(location, name, description)
            else:
                generate_playing_card(location, name, None)

            locations.at[index, 'Generated'] = True
            locations.to_csv(LOCATIONS_FILE, index=False)

generate_all_playing_cards()


## Reset generation progress

In [ ]:
def reset_generated_locations():
    # Read csv file
    locations = pd.read_csv(LOCATIONS_FILE)

    # Update 'Generated' column to False for all rows
    locations['Generated'] = False

    # Save updated DataFrame to csv file
    locations.to_csv(LOCATIONS_FILE, index=False)

# Uncomment and execute if you want to reset, which cards have been generated
# reset_generated_locations()